In [2]:
import awswrangler as wr
import os
import pandas as pd

## Pipeline parameters

In [3]:
pipeline_output_path = "s3://genomics-workflow-core/Results/GenomeMining"
project = "IMG"
prefix = "20220707"
seedfile_name = "test_seedfile.csv"

input_s3_path = "s3://maf-versioned/GenomeMining/Genomes/IMG/"
file_extension = "tar.gz"

seedfile = os.path.join(pipeline_output_path, project, prefix, seedfile_name)

In [4]:
file_paths = wr.s3.list_objects(input_s3_path, file_extension)

In [5]:
len(file_paths)

78357

In [6]:
file_paths_test = file_paths[:5]
file_paths_test

['s3://maf-versioned/GenomeMining/Genomes/IMG/2022827000.tar.gz',
 's3://maf-versioned/GenomeMining/Genomes/IMG/2040502012.tar.gz',
 's3://maf-versioned/GenomeMining/Genomes/IMG/2140918011.tar.gz',
 's3://maf-versioned/GenomeMining/Genomes/IMG/2228664006.tar.gz',
 's3://maf-versioned/GenomeMining/Genomes/IMG/2228664007.tar.gz']

In [7]:
seedfile_df = pd.DataFrame({
"genome_id" : [os.path.basename(filename).replace(".tar.gz","") for filename in file_paths_test],
"genome_path" : file_paths_test
})

wr.s3.to_csv(df=seedfile_df, path=seedfile, index=False)

{'paths': ['s3://genomics-workflow-core/Results/GenomeMining/IMG/20220707/test_seedfile.csv'],
 'partitions_values': {}}